In [5]:
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from keras.utils import to_categorical
import pandas as pd

In [28]:
minmax = pd.read_csv('C:/Users/brook/Documents/MSDS/MSDS 411/Assignment 3/assign3_minmax.csv')
minmax.head()
min_cop = minmax.copy()
y = minmax['credit.class']
minmax = minmax.drop(columns = {"Unnamed: 0", 'credit.class'})

,Unnamed: 0,credit.class,checking_status0_X_200,checking_status_200,checking_statusno_checking,duration,credit_historyall_paid,credit_historyexisting_paid,credit_historydelayed_previously,credit_historycritical_other_existing_credit,...,other_payment_plansstores,other_payment_plansnone,housingown,housingforfree,existing_credits,jobunskilled_resident,jobskilled,jobhigh_qualif_self_emp_mgmt,num_dependents,own_telephoneyes
0,1,good,0,0,0,0.029412,0,0,0,1,...,0,1,1,0,0.333333,0,1,0,0,1
1,2,bad,0,0,0,0.029412,0,0,0,1,...,0,1,1,0,0.333333,0,1,0,0,1
2,3,good,0,0,0,0.029412,0,0,0,1,...,0,1,1,0,0.333333,0,1,0,0,1
3,4,good,0,0,0,0.029412,0,0,0,1,...,0,1,1,0,0.333333,0,1,0,0,1
4,5,bad,0,0,0,0.029412,0,0,0,1,...,0,1,1,0,0.333333,0,1,0,0,1


In [29]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_ground,valid_ground = train_test_split(minmax,
 minmax,
 test_size=0.2,
 random_state=13)
train_X.shape

(800000, 44)

In [45]:
import keras
from keras import layers

# This is the size of our encoded representations
encoding_dim = 25  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

input_img = keras.Input(shape =  (44,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)

encoder = keras.Model(input_img, encoded)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(44, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [46]:
# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [47]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [48]:
autoencoder.fit(train_X, train_X,
                epochs=50,
                batch_size=5000,
                shuffle=True,
                validation_data=(valid_X, valid_X))

Train on 800000 samples, validate on 200000 samples
Epoch 1/50
800000/800000 [==============================] - 4s 5us/step - loss: 0.5002 - val_loss: 0.3814
Epoch 2/50
800000/800000 [==============================] - 4s 5us/step - loss: 0.3216 - val_loss: 0.2675
Epoch 3/50
800000/800000 [==============================] - 4s 5us/step - loss: 0.2273 - val_loss: 0.1927
Epoch 4/50
800000/800000 [==============================] - 4s 5us/step - loss: 0.1689 - val_loss: 0.1483
Epoch 5/50
800000/800000 [==============================] - 4s 5us/step - loss: 0.1336 - val_loss: 0.1206
Epoch 6/50
800000/800000 [==============================] - 4s 5us/step - loss: 0.1112 - val_loss: 0.1027
Epoch 7/50
800000/800000 [==============================] - 5s 6us/step - loss: 0.0966 - val_loss: 0.0909
Epoch 8/50
800000/800000 [==============================] - 5s 7us/step - loss: 0.0868 - val_loss: 0.0829
Epoch 9/50
800000/800000 [==============================] - 4s 5us/step - loss: 0.0801 - val_loss: 0

In [49]:
autoencoder = Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())
autoencoder.summary()

Model: "model_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 44)                0         
_________________________________________________________________
model_19 (Model)             (None, 25)                1125      
_________________________________________________________________
model_21 (Model)             (None, 44)                1144      
Total params: 2,269
Trainable params: 2,269
Non-trainable params: 0
_________________________________________________________________


In [62]:
%load_ext tensorboard
import tensorflow as tf
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
from keras.callbacks import TensorBoard
# Load the TensorBoard notebook extension

autoencoder.fit(train_X, train_X,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(valid_X, valid_X),
                callbacks=[TensorBoard(log_dir='/tmp/logs_fit/')])

Train on 800000 samples, validate on 200000 samples
Epoch 1/10
800000/800000 [==============================] - 70s 88us/step - loss: 1.3488e-04 - val_loss: 1.3391e-04
Epoch 2/10
800000/800000 [==============================] - 72s 90us/step - loss: 1.3472e-04 - val_loss: 1.3198e-04
Epoch 3/10
800000/800000 [==============================] - 73s 91us/step - loss: 1.3455e-04 - val_loss: 1.4533e-04ss: 1.3455e- - ETA: 0s - loss
Epoch 4/10
800000/800000 [==============================] - 72s 90us/step - loss: 1.3444e-04 - val_loss: 1.4347e-04
Epoch 5/10
800000/800000 [==============================] - 74s 93us/step - loss: 1.3420e-04 - val_loss: 1.3534e-04
Epoch 6/10
800000/800000 [==============================] - 68s 85us/step - loss: 1.3407e-04 - val_loss: 1.3389e-04A: 1s - loss: 1.3411e- - ETA: 1s - loss: 1 - ETA: 1s -
Epoch 7/10
800000/800000 [==============================] - 70s 88us/step - loss: 1.3400e-04 - val_loss: 1.3400e-04
Epoch 8/10
800000/800000 [===========================

In [71]:
%tensorboard --logdir autoencoder


Reusing TensorBoard on port 6006 (pid 14816), started 0:16:22 ago. (Use '!kill 14816' to kill it.)